In [1]:
import ast
import time
from pathlib import Path
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

import torch
import logging
from logging import getLogger
from recbole.config import Config
from recbole.data import create_dataset, data_preparation
from recbole.utils import init_seed, init_logger
from recbole.quick_start import run_recbole

2024-02-09 18:38:21.341294: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
DATA_PATH = Path("/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/kion_train")
users_df = pd.read_csv(DATA_PATH / 'users.csv')
items_df = pd.read_csv(DATA_PATH / 'items.csv')
interactions_df = pd.read_csv(DATA_PATH / 'interactions.csv')

In [5]:
interactions_df['t_dat'] = pd.to_datetime(interactions_df['last_watch_dt'], format="%Y-%m-%d")
interactions_df['timestamp'] = interactions_df.t_dat.values.astype(np.int64) // 10 ** 9

In [6]:
df = interactions_df[['user_id', 'item_id', 'timestamp']].rename(
    columns={'user_id': 'user_id:token', 'item_id': 'item_id:token', 'timestamp': 'timestamp:float'})

In [7]:
!mkdir recbox_data

In [8]:
df.to_csv('recbox_data/recbox_data.inter', index=False, sep='\t')

In [9]:
df.head(3)

,user_id:token,item_id:token,timestamp:float
0,176549,9506,1620691200
1,699317,1659,1622246400
2,656683,7107,1620518400


In [10]:
parameter_dict = {
    'data_path': '',
    'USER_ID_FIELD': 'user_id',
    'ITEM_ID_FIELD': 'item_id',
    'TIME_FIELD': 'timestamp',
    'device': 'CPU',
    'user_inter_num_interval': "[40,inf)",
    'item_inter_num_interval': "[40,inf)",
    'load_col': {'inter': ['user_id', 'item_id', 'timestamp']},
    'neg_sampling': None,
    'epochs': 10,
    'eval_args': {
        'split': {'RS': [9, 0, 1]},
        'group_by': 'user',
        'order': 'TO',
        'mode': 'full'}
}
config = Config(model='MultiVAE', dataset='recbox_data', config_dict=parameter_dict)

init_seed(config['seed'], config['reproducibility'])

init_logger(config)
logger = getLogger()
c_handler = logging.StreamHandler()
c_handler.setLevel(logging.INFO)
logger.addHandler(c_handler)

In [11]:
dataset = create_dataset(config)
logger.info(dataset)

09 Feb 17:08    INFO  recbox_data
The number of users: 13355
Average actions of users: 63.815710648494836
The number of items: 3294
Average actions of items: 258.78985727300335
The number of inters: 852195
The sparsity of the dataset: 98.06281322904924%
Remain Fields: ['user_id', 'item_id', 'timestamp']
recbox_data
The number of users: 13355
Average actions of users: 63.815710648494836
The number of items: 3294
Average actions of items: 258.78985727300335
The number of inters: 852195
The sparsity of the dataset: 98.06281322904924%
Remain Fields: ['user_id', 'item_id', 'timestamp']


In [12]:
train_data, valid_data, test_data = data_preparation(config, dataset)

09 Feb 17:08    INFO  [Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
[Training]: train_batch_size = [2048] train_neg_sample_args: [{'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}]
09 Feb 17:08    INFO  [Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]
[Evaluation]: eval_batch_size = [4096] eval_args: [{'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}]


In [13]:
model_list = ['MultiVAE', 'CDAE', 'BPR', 'MultiDAE']

for model_name in model_list:
    print(f"running {model_name}...")
    start = time.time()
    result = run_recbole(model=model_name, dataset = 'recbox_data',config_dict = parameter_dict)
    t = time.time() - start
    print(f"It took {t/60:.2f} mins")
    print(result)

running MultiVAE...


09 Feb 17:13    INFO  ['/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/.venv/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/Users/tanchik/Library/Jupyter/runtime/kernel-9cfb4bcb-80d1-441b-994e-bfc4c26c4c5b.json']
['/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/.venv/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/Users/tanchik/Library/Jupyter/runtime/kernel-9cfb4bcb-80d1-441b-994e-bfc4c26c4c5b.json']
09 Feb 17:13    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
e

It took 1.41 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.084), ('mrr@10', 0.1695), ('ndcg@10', 0.0825), ('hit@10', 0.3503), ('precision@10', 0.0467)])}
running CDAE...


09 Feb 17:15    INFO  ['/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/.venv/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/Users/tanchik/Library/Jupyter/runtime/kernel-9cfb4bcb-80d1-441b-994e-bfc4c26c4c5b.json']
['/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/.venv/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/Users/tanchik/Library/Jupyter/runtime/kernel-9cfb4bcb-80d1-441b-994e-bfc4c26c4c5b.json']
09 Feb 17:15    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
e

It took 1.10 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0634), ('mrr@10', 0.1439), ('ndcg@10', 0.0659), ('hit@10', 0.2723), ('precision@10', 0.0344)])}
running BPR...


09 Feb 17:16    INFO  ['/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/.venv/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/Users/tanchik/Library/Jupyter/runtime/kernel-9cfb4bcb-80d1-441b-994e-bfc4c26c4c5b.json']
['/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/.venv/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/Users/tanchik/Library/Jupyter/runtime/kernel-9cfb4bcb-80d1-441b-994e-bfc4c26c4c5b.json']
09 Feb 17:16    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
e

It took 1.85 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0824), ('mrr@10', 0.1716), ('ndcg@10', 0.0819), ('hit@10', 0.3479), ('precision@10', 0.0457)])}
running MultiDAE...


09 Feb 17:18    INFO  ['/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/.venv/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/Users/tanchik/Library/Jupyter/runtime/kernel-9cfb4bcb-80d1-441b-994e-bfc4c26c4c5b.json']
['/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/.venv/lib/python3.9/site-packages/ipykernel_launcher.py', '-f', '/Users/tanchik/Library/Jupyter/runtime/kernel-9cfb4bcb-80d1-441b-994e-bfc4c26c4c5b.json']
09 Feb 17:18    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
e

It took 1.18 mins
{'best_valid_score': -inf, 'valid_score_bigger': True, 'best_valid_result': None, 'test_result': OrderedDict([('recall@10', 0.0829), ('mrr@10', 0.1655), ('ndcg@10', 0.081), ('hit@10', 0.3438), ('precision@10', 0.0459)])}


In [3]:
from recbole.quick_start import load_data_and_model

config, model, dataset, train_data, valid_data, test_data = load_data_and_model(
    model_file='/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/service/api/recsys/models/MultiVAE-Feb-09-2024_17-14-29.pth',
)


09 Feb 18:38    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separa

In [16]:
class MultiVAE:
    def __init__(self, path: str, ):
        self.config, self.model, self.dataset, self.train_data, self.valid_data, self.test_data = load_data_and_model(
            model_file=path)
        self.popular = np.load('/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/service/api/recsys/models/popular.npy')

    def add_popular(self, item_ids: np.ndarray, N: int) -> np.ndarray:
        mask = ~np.isin(self.popular, item_ids)
        filtered_popular = self.popular[mask]
        combined = np.concatenate([item_ids, filtered_popular])
        combined = combined.astype(int)
        result = combined[:N]
        return result

    def recommend(self, user_id: int, n: int) -> list:
        try:
            uid_series = self.dataset.token2id(self.dataset.uid_field, [str(user_id)])
        except ValueError:
            recos = self.popular[:n].tolist()
            return recos

        interaction = {self.model.USER_ID: torch.tensor([uid_series])}
        scores = self.model.full_sort_predict(interaction)
        recommended_items = torch.argsort(scores, descending=True)[:n]
        recos = self.dataset.id2token(self.dataset.iid_field, recommended_items)

        if len(recos) < n:
            recos = self.add_popular(recos, n)
        recos = [int(item) for item in recos]
        return recos

In [17]:
popular = np.load('/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/service/api/recsys/models/popular.npy')

In [18]:
model = MultiVAE('/Users/tanchik/Desktop/Настоящее/учеба/RecSys/RecoServiceTemplate/service/api/recsys/models/MultiVAE-Feb-09-2024_17-14-29.pth')

09 Feb 18:44    INFO  
General Hyper Parameters:
gpu_id = 0
use_gpu = True
seed = 2020
state = INFO
reproducibility = True
data_path = recbox_data
checkpoint_dir = saved
show_progress = True
save_dataset = False
dataset_save_path = None
save_dataloaders = False
dataloaders_save_path = None
log_wandb = False

Training Hyper Parameters:
epochs = 10
train_batch_size = 2048
learner = adam
learning_rate = 0.001
train_neg_sample_args = {'distribution': 'uniform', 'sample_num': 1, 'alpha': 1.0, 'dynamic': False, 'candidate_num': 0}
eval_step = 1
stopping_step = 10
clip_grad_norm = None
weight_decay = 0.0
loss_decimal_place = 4

Evaluation Hyper Parameters:
eval_args = {'split': {'RS': [9, 0, 1]}, 'order': 'TO', 'group_by': 'user', 'mode': {'valid': 'full', 'test': 'full'}}
repeatable = False
metrics = ['Recall', 'MRR', 'NDCG', 'Hit', 'Precision']
topk = [10]
valid_metric = MRR@10
valid_metric_bigger = True
eval_batch_size = 4096
metric_decimal_place = 4

Dataset Hyper Parameters:
field_separa

In [19]:
model.recommend(176549, 10)

[9728, 13865, 10440, 11754, 15464, 1785, 4436, 4457, 14741, 5693]